# Problem Set 5

**Introduction:**
#### Regularisation involves shrinking the estimated coefficients towards zero relative to least  square estimates. By shrinking the estimated coefficients we substantially reduce the variance at the cost of negligible increase in bias.
#### There are two shrinkage techniques: Ridge regression and LASSO



In [10]:
rm(list = ls())
set.seed(123)

##############################Required packages#################################

library(glmnet)
library(ggplot2)
library(plotly)
library(mvtnorm)
library(ggpubr)
require(reshape2)
library(patchwork)

####################################Given Parameters and DGP####################

n <- 100
p <- 3
beta <- c(0.5, 0.5, -0.5)
eps_sd<- sqrt(10)
x_mu <- rep(0,p)
var <- matrix(c(16,0,1.38, 0,116,0.04, 1.38,0.04,18), ncol=3)

# DGP without intercept
eps <- rnorm(n, 0, eps_sd)
x <- rmvnorm(n, x_mu, var) 
y <- x%*%beta + eps

# Test sample and training sample
dt = sort(sample(nrow(x), nrow(x)*0.5))
x_train <- x[dt, ]
x_test  <- x[-dt,]
y_train <- y[dt, ]
y_test  <- y[-dt,]

ERROR: Error in library(plotly): there is no package called 'plotly'


**Exercise 1, a**

Write a function to calculate ridge regression estimator in closed form:

#### Ridge regression: Ridge regression estimates are values that minimizes 

$ \hat{\beta}_{ridge} = \arg \min_{b_0, ..., b_K} \sum_{i=1}^{n} ( Y_i - b_0 - X_{i1}b_1 - ... -  X_{ik}b_k )^2 + \lambda \sum_{k = 0}^{K} b_k^2  \\
 = \Bigg( \sum_{i=1}^{n} X_i X_{i}^{'} + \lambda I_{ (K+1) \times (K+1) } \Bigg ) ^{-1}  \sum_{i = 1}^{n} X_iY_i $

for $Y_i = \beta_0 + X_{i1} \beta_1 +  X_{i2} \beta_2 + ...+ X_{ik} \beta_k + U_i $, 



In [ ]:

######################## QUESTION A ############################################

func_ridge <- function(lambda, x, y){
  
  if(sd(x[,1])==0){
    x <- cbind(x[,1], scale(x[,2:3]))
  } else {
    x <- scale(x)
    y <- scale(y)
  }
  
  b_ridge <-solve((t(x)%*%x+lambda*diag(p)))%*% t(x)%*%y
  
  return(b_ridge)
}


**Exercise 1, b**

Specify a grid of 100 lambda values and find ridge reg coef and plot the coef paths:

In [ ]:
#############################Question B ########################################

m <- 100
lambda <- seq(0 , 500, length = m)
store_ridge <- matrix(NaN, m, p)

for(i in 1:m){
  store_ridge[i,] <- func_ridge(lambda[i], x=x_train, y=y_train)
}

round(head(store_ridge), 3)
round(tail(store_ridge), 3)

###Plot 1 - Coefficient paths 
Result1 <- data.frame(cbind(store_ridge,lambda, ld = round(lambda, digits=0)))
options(warn = -1)
graph1 <- ggplot(data = Result1, aes(x = lambda)) +
  geom_hline(yintercept = 0, linetype="dashed")+
  geom_point(aes(y = store_ridge[,1], colour = "beta1", frame = ld)) + geom_line(aes(y =store_ridge[,1], colour = "beta1")) +
  geom_point(aes(y = store_ridge[,2], colour = 'beta2', frame = ld)) + geom_line(aes(y = store_ridge[,2], colour = "beta2")) +
  geom_point(aes(y = store_ridge[,3], colour = "beta3", frame = ld)) + geom_line(aes(y = store_ridge[,3], colour = "beta3")) +
  labs(title = 'Path of coefficients', x = 'Lambda', y = 'Standardized Coefficients') +
  scale_colour_manual("coefficients", breaks = c("beta1", "beta2", "beta3"), values = c( "red", "blue", "green")) +  
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

ggplotly(graph1)


**Exercise 1, c** 

Generate a test sample of same size and calc prediction and training error for each  lambda and OLS estimate. Show results in graph



In [ ]:
################################### Question C ##################################

store_errors <- matrix(NaN, m, 3)

for(i in 1:m){
  
  store_errors[i,2] <- mean((y_test - x_test %*% store_ridge[i,])^2)
  store_errors[i,3] <- mean((y_train - x_train %*% store_ridge[i,])^2)
  
}
store_errors <- data.frame("Lambda" = lambda, "Prediction Error" = store_errors[,2], "Training Error" = store_errors[,3])
round(head(store_errors), 3)
round(tail(store_errors), 3)

# Plot 2 - Prediction and training error

Result_errors <- data.frame(cbind(store_errors, lambda, ld = round(lambda, digits=0)))
options(warn= -1)
graph_error <- ggplot(data = Result_errors, aes(x = lambda)) +
  geom_point(aes(y =store_errors[,2], colour = "Prediction error", frame = ld)) + geom_line(aes(y =store_errors[,2], colour = "Prediction error")) +
  geom_point(aes(y =store_errors[,3], colour = 'Training error', frame = ld)) + geom_line(aes(y =store_errors[,3], colour = "Training error")) +
  labs(title = 'Path of Training and Test errors', x = 'Lambda', y = 'Errors') +
  scale_colour_manual("coefficients", breaks = c("Prediction error", "Training error"),
                      values = c( "red", "blue"))   +  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
                                                             panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

ggplotly(graph_error)

**Exercise 1, d** 

Replace one of the covariates to -10. Show the coefficient paths graphically.

In [ ]:
###################################### Question D ###############################


#DGP with a constant
x_cons   <- cbind(rep(-10,n), x[,2:3])
y_cons   <- x_cons%*%beta + eps

#Training data
x_train_cons <- x_cons[dt, ]
y_train_cons <- y_cons[dt, ]

store_ridge_cons <- matrix(NaN, m, p)

for(i in 1:m){
  store_ridge_cons[i,] <- func_ridge(lambda[i], x=x_train_cons, y=y_train_cons)
}

round(head(store_ridge), 4)
round(head(store_ridge_cons), 4)


###################### Plot 3 - COefficient path_ new and old ###################
Result2 <- data.frame(cbind(store_ridge_cons, lambda))
graph2 <- ggplot(data = Result2, aes(x = lambda)) +
  geom_hline(yintercept = 0, linetype="dashed")+
  geom_line(aes(y =store_ridge[,2], colour = "beta2")) + geom_line(aes(y =store_ridge_cons[,2], colour = "beta2_new")) +
  labs(title = 'Path of coefficients', x = 'Lambda', y = 'Standardized Coefficients') +
  scale_colour_manual("coefficients", breaks = c("beta2", "beta2_new"),
                      values = c( "blue", "red"))   +  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
                                                             panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

graph3 <- ggplot(data = Result2, aes(x = lambda)) +
  geom_hline(yintercept = 0, linetype="dashed")+
  geom_line(aes(y =store_ridge[,1], colour = "beta1")) + geom_line(aes(y =store_ridge_cons[,1], colour = "beta1_new")) +
  labs(title = 'Path of coefficients', x = 'Lambda', y = 'Standardized Coefficients') +
  scale_colour_manual("coefficients", breaks = c("beta1", "beta1_new"),
                      values = c( "blue", "red"))   +  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
                                                             panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

graph4 <- ggplot(data = Result2, aes(x = lambda)) +
  geom_hline(yintercept = 0, linetype="dashed")+
  geom_line(aes(y =store_ridge[,3], colour = "beta3")) + geom_line(aes(y =store_ridge_cons[,3], colour = "beta3_new")) +
  labs(title = 'Path of coefficients', x = 'Lambda', y = 'Standardized Coefficients') +
  scale_colour_manual("coefficients", breaks = c("beta3", "beta3_new"),
                      values = c( "blue", "red"))   +  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
                                                             panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

figure <- ggarrange(graph2, graph3, graph4, ncol =2, nrow = 2)
figure

**Exercise 1, e** 

Use glmnet package and determine optimal lambda from 10-fold cross-validation error


In [ ]:
################################## Question e ####################################

cross <- cv.glmnet(x_train, y_train, alpha=0, nfolds=10)
bestlam = cross$lambda.min
bestlam 

###### Plot 4 Mean Cross Validation Error
Result4 <- data.frame(cbind(cross$lambda, cross$cvm, ld = round(cross$lambda, digits=0)))
options(warn = -1)
graph7 <- ggplot(data = Result4, aes(x = cross$lambda)) +
  geom_vline(xintercept = bestlam, linetype="dashed") +
  geom_point(aes(y = cross$cvm, colour = "cvm", frame = ld)) + geom_line(aes(y = cross$cvm, colour = "cvm")) +
  labs(title = 'Path of Mean CV error', x = 'Lambda', y = 'CV error') +
  scale_colour_manual("", breaks ="cvm", values = "red") +  
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), plot.title = element_text(hjust = 0.5), axis.line = element_line(colour = "black"))

ggplotly(graph7)

**Exercise 2, a** 

Compare the performance of the OLS estimator and the ridge regression estimator in terms of the prediction
error for 100 simulation runs

In [ ]:

rm(list = ls())

# Set sample size and number of simulations
n = 100
n_sims = 100

# Set number of regressors (p), true Parameters (beta) and variance of error
p = 3
beta = c(0.5,0.5,-0.5)
esp_sd = 10**0.5
esp = rnorm(n,0,esp_sd)

#Define var-cov matrix

cov_mat <- function(m) {
    A <- matrix(runif(m^2)*2-1, ncol=m) 
    Sigma <- t(A) %*% A
  return(Sigma)
  }

# Set out-of-sample Data

x_mat_oos = rmvnorm(n, rep(0,p), cov_mat(p))
Y_oos = x_mat_oos%*%beta + esp

predict_error = data.frame(matrix(ncol = 2, nrow = 0)) 
colnames(predict_error) = c("OLS", "Ridge")

# Conduct the simulation for 100 rounds, in each calculate estimated parameters

for (i_sims in 1:n_sims) {
  x_mat = rmvnorm(n, rep(0,p), cov_mat(p))
  Y = x_mat%*%beta + esp

  # Determine the optimal value of lambda using cross validation

  lambda = cv.glmnet(x_mat, Y, alpha = 0, nfolds = 10)$lambda.min

  # Estimate coefficients for Regression and Ridge 

  coefs_est_ridge = c(solve(t(x_mat) %*% x_mat + lambda*diag(p)) %*% t(x_mat) %*% Y)
  coefs_est_ols = c(solve(t(x_mat) %*% x_mat) %*% t(x_mat) %*% Y)

  # Now calculate the  prediction Error 
  
  res_predict_ols = Y_oos - x_mat_oos%*%coefs_est_ols
  s2_predict_ols = sum(res_predict_ols^2)/(n) 
  res_predict_ridge = Y_oos - x_mat_oos%*%coefs_est_ridge
  s2_predict_ridge = sum(res_predict_ridge^2)/(n) 

  #Update the data frame of predicted errors 

  predict_error[i_sims,] = c(s2_predict_ols, s2_predict_ridge)

}

#Mean Prediction Error 

average_pred_error = colMeans(predict_error[sapply(predict_error, is.numeric)]) 


**Exercise 2, b** 

Propose a manipulation of the dgp that will improve the ridge regression performance relative to the OLS
estimate.

In [ ]:
rm(list = ls())

# Set correlated variance and covariance matrix

cov_mat_corr <- function(m) {
    A <- matrix(runif(m^2)*2-1, ncol=m) 
    Sigma <- t(A) %*% A
  return(Sigma)
}

# Set uncorrelated variance and covariance matrix

cov_mat_uncorr <- function(m) {
  return(diag(m))
}


# The function ad_ridge_ols takes number of simulations, number of observations, 
# number of features, variance of error, and 
# returns prediction error disaggregated by OLS and Ridge.  

ad_ridge_ols <- function(n_sims,n,p,var,covvarmat){

  beta = runif(p, min= -1, max=1)
  esp_sd = var**0.5
  predict_error = data.frame(matrix(ncol = 2, nrow = 0)) 
 
# Conduct Simulation for 100 rounds 

  for (i_sims in 1:n_sims) {
    esp = rnorm(n,0,esp_sd)
    x_mat = rmvnorm(n, rep(0,p), covvarmat)
    Y = x_mat%*%beta + esp
    
  # Draw Test Sample using identical distribution 

    x_mat_oos = rmvnorm(n, rep(0,p), covvarmat)
    Y_oos = x_mat_oos%*%beta + esp

  # Use the optimal Lambda given by  10 fold cross-validation 

    lambda = cv.glmnet(x_mat, Y, alpha = 0, nfolds = 10)$lambda.min

  # Estimating OLS and Ridge Coefficients
    
    coefs_est_ridge = c(solve(t(x_mat) %*% x_mat + lambda*diag(p)) %*% t(x_mat) %*% Y)
    coefs_est_ols = c(solve(t(x_mat) %*% x_mat) %*% t(x_mat) %*% Y)

    # Prediction Error 
    res_predict_ols = Y_oos - x_mat_oos%*%coefs_est_ols
    s2_predict_ols = sum(res_predict_ols^2)/(n) 
    res_predict_ridge = Y_oos - x_mat_oos%*%coefs_est_ridge
    s2_predict_ridge = sum(res_predict_ridge^2)/(n) 
    # Update the values 
    predict_error[i_sims,] = c(s2_predict_ols, s2_predict_ridge)
    }
  return(colMeans(predict_error[sapply(predict_error, is.numeric)]))
}

# The function takes max number of features and maps prediction errors when 
# number of features range from 3 to the set max number of features.  

features_ridge_ols <- function(p){ 
  feature_pred_error = data.frame(matrix(ncol = 3, nrow = 0)) 
  colnames(feature_pred_error) = c("Features", "OLS", "Ridge")
  feature_l_seq = seq(3, p, length = 30) 
  for (i_p in floor(feature_l_seq)) {
    feature_pred_error[i_p,] =  c(i_p, ad_ridge_ols(n_sims = 100, n = 100, i_p, var = 10, cov_mat_uncorr(i_p)))
    }
  return(feature_pred_error[complete.cases(feature_pred_error),])
}

prediction_errors = features_ridge_ols(90)

# Plot when features are changed from 3 to 90.

df_prediction_errors = melt(prediction_errors ,  id.vars = 'Features', variable.name = 'Types')
plot_prediction_errors = ggplot(df_prediction_errors, aes(Features,value)) + 
                         geom_line(aes(colour = Types)) + 
                         ylab("Prediction Erros") + 
                         ggtitle("Evolution of prediction errors with increase in number of predictors") + 
                         xlab(" Number of Features") 

print(plot_prediction_errors)

# Change the variance of error, take only 10 values in the range. 

var_ridge_ols <- function(v){ 
  var_pred_error = data.frame(matrix(ncol = 3, nrow = 0)) 
  var_all = seq(from = 1, to = v,  length = 10)
  p = 25
  colnames(var_pred_error) = c("Variance", "OLS", "Ridge")
  for (i_v in var_all) {
    var_pred_error[i_v,] =  c(i_v, ad_ridge_ols(n_sims = 100, n = 100, p, var = i_v,cov_mat_uncorr(p)))
    }
  return(var_pred_error[complete.cases(var_pred_error),])
}

# We allow variance to range from 1 to 100, and work with 10 values in between

change_variance_errors = var_ridge_ols(100)

#  Now Draw the Graphs

df_change_variance_errors = melt(change_variance_errors ,  id.vars = 'Variance', variable.name = 'Types')
plot_change_variance_errors = ggplot(df_change_variance_errors, aes(Variance,value)) + 
                         geom_line(aes(colour = Types)) + 
                         ylab("Prediction Errors") + 
                         ggtitle("Evolution of prediction errors with increase in Variance for error term") + 
                         xlab("Variance of Error Term") 

print(plot_change_variance_errors)

#  Now Test the third data generation Process, which looks into
#  effects of changing the variance and covariance matrix
#  use n = 100, n_sims = 100, p = 90

iid_ridge_ols <- function(p){ 
  iid_pred_error = data.frame(matrix(ncol = 3, nrow = 0)) 
  colnames(iid_pred_error) = c("Features", "OLS", "Ridge")
  feature_l_seq = seq(3, p, length = 30) 
  for (i_p in floor(feature_l_seq)) {
    iid_pred_error[i_p,] =  c(i_p, ad_ridge_ols(n_sims = 100, n = 100, i_p, var = 10, cov_mat_uncorr(i_p)))
    }
  return(iid_pred_error[complete.cases(iid_pred_error),])
}

iid_prediction_errors = iid_ridge_ols(90)


# The following function predicts error when x-variables are correlated. 

crr_ridge_ols <- function(p){ 
  cid_pred_error = data.frame(matrix(ncol = 3, nrow = 0)) 
  colnames(cid_pred_error) = c("Features", "OLS", "Ridge")
  feature_l_seq = seq(3, p, length = 30) 
  for (i_p in floor(feature_l_seq)) {
    cid_pred_error[i_p,] =  c(i_p, ad_ridge_ols(n_sims = 100, n = 100, i_p, var = 10,cov_mat_corr(i_p)))
    }
  return(cid_pred_error[complete.cases(cid_pred_error),])
}

cr_prediction_errors = crr_ridge_ols(90)

#  Graphs g_corr refers to coorrelated and g_iid for uncoorelated x-variables. In both cases, 
#  we map prediction error with number of features. 


df_g_corr = melt(cr_prediction_errors ,  id.vars = 'Features', variable.name = 'Types')
plot_g_corr = ggplot(df_g_corr, aes(Features,value)) + 
                         geom_line(aes(colour = Types)) + 
                         ylab("Prediction Errors") + 
                         ggtitle("Evolution of prediction errors with increase in  number of correlated predictors") + 
                         xlab("# Number of Features") 

print(plot_g_corr)


df_g_uncorr = melt(iid_prediction_errors ,  id.vars = 'Features', variable.name = 'Types')
plot_g_uncorr = ggplot(df_g_uncorr, aes(Features,value)) + 
                         geom_line(aes(colour = Types)) + 
                         ylab("Prediction Errors") + 
                         ggtitle("Evolution of prediction errors with increase in  number of uncorrelated predictors") + 
                         xlab("# Number of Features") 

print(plot_g_uncorr)

